In [ ]:
import os
import sys

os.chdir("../")  # cambiar el directorio de trabajo a la raíz del repo
sys.path.append("./")  # add current dir to module search path

from data_exploration.utils import summarize_points_gt  # noqa: E402

## Adaptación de header de CSV con anotaciones

Fue necesario hacer un pequeño arreglo de los encabezados de los csv con anotaciones estilo `bbox` para
que fueran usables como entradas de

```bash
sed '1s/Image,x1,y1,x2,y2,Label/images,x_min,y_min,x_max,y_max,labels/'
data/groundtruth/csv/train_big_size_A_B_E_K_WH_WB.csv
> data/gt-preprocessed/csv/train_big_size_A_B_E_K_WH_WB-fixed-header.csv

sed '1s/Image,x1,y1,x2,y2,Label/images,x_min,y_min,x_max,y_max,labels/'
data/groundtruth/csv/val_big_size_A_B_E_K_WH_WB.csv
> data/gt-preprocessed/csv/val_big_size_A_B_E_K_WH_WB-fixed-header.csv
```

In [ ]:
import pandas as pd

# Ejemplo de los archivos antes
pd.read_csv("data/groundtruth/csv/train_big_size_A_B_E_K_WH_WB.csv")

In [21]:
pd.read_csv("data/gt-preprocessed/csv/train_big_size_A_B_E_K_WH_WB-fixed-header.csv")

,images,x_min,y_min,x_max,y_max,labels
0,L_07_05_16_DSC00126.JPG,2488,357,2520,427,3
1,L_07_05_16_DSC00126.JPG,1707,446,1746,471,3
2,L_07_05_16_DSC00127.JPG,2612,2314,2641,2361,3
3,L_07_05_16_DSC00127.JPG,1793,2450,1843,2475,3
4,L_07_05_16_DSC00150.JPG,1492,1381,1586,1441,2
...,...,...,...,...,...,...
6957,2f75ff230b98fc9c58d5aa099e0da4b4ca5350eb.JPG,2975,3211,3003,3246,6
6958,2f75ff230b98fc9c58d5aa099e0da4b4ca5350eb.JPG,2986,3439,3019,3490,6
6959,2f75ff230b98fc9c58d5aa099e0da4b4ca5350eb.JPG,1811,993,1842,1033,6
6960,2f75ff230b98fc9c58d5aa099e0da4b4ca5350eb.JPG,1425,993,1468,1079,6


Esto comprueba que los datos no se vieron afectados sino solo los encabezados.

### Creación de conjuntos de parches

Los siguientes comandos se usaron para crear parches para los conjuntos de entrenamiento y validación.
También se ejecutó en el mismo paso el script `oficial_herdnet/tools/csv_to_points.py`  para convertir
anotaciones tipo `bbox` a anotaciones tipo puntos `xy`

Principales parámetros:

```python
size = 512
overlap = 160
min_visibility = 0.1
```


```bash
!dvc repro -s patches-512-ol-160-mv01-train
!dvc repro -s patches-512-ol-160-mv01-val

# Comandos de python ejecutados por dvc:

python oficial_herdnet/tools/patcher.py data/train 512 512 160
       data/patches-512-ol-160-m01/train
       -csv data/groundtruth/csv/train_big_size_A_B_E_K_WH_WB-fixed-header.csv
       && python oficial_herdnet/tools/csv_to_points.py
       -i data/patches-512-ol-160-m01/train/gt.csv
       -o data/patches-512-ol-160-m01/train_points_gt.csv

python oficial_herdnet/tools/patcher.py data/val 512 512 160
      data/patches-512-ol-160-m01/val
      -csv data/groundtruth/csv/val_big_size_A_B_E_K_WH_WB-fixed-header.csv
      && python oficial_herdnet/tools/csv_to_points.py
      -i data/patches-512-ol-160-m01/val/gt.csv
      -o data/patches-512-ol-160-m01/val_points_gt.csv

```

In [12]:
# Número de parches en entrenamiento y validación resp:
!ls -l data/patches-512-ol-160-m01/train/*.JPG | wc -l
!ls -l data/patches-512-ol-160-m01/val/*.JPG | wc -l

8581
1166


In [14]:
!wc -l data/patches-512-ol-160-m01/train_points_gt.csv

16105 data/patches-512-ol-160-m01/train_points_gt.csv


In [49]:
summarize_points_gt("data/patches-512-ol-160-m01/train_points_gt.csv")

archivo: data/patches-512-ol-160-m01/train_points_gt.csv

16104 filas - 4 columnas: [images, x, y, labels]

## Número de valores únicos por columna:
|        |    0 |
|:-------|-----:|
| images | 8581 |
| x      |  510 |
| y      |  511 |
| labels |    6 |

## Conteos de nulos por columna:
|    | index   |   0 |
|---:|:--------|----:|
|  0 | images  |   0 |
|  1 | x       |   0 |
|  2 | y       |   0 |
|  3 | labels  |   0 |

## Estadísticas de coordenadas de puntos (x, y)
|       |         x |         y |
|:------|----------:|----------:|
| count | 16104     | 16104     |
| mean  |   252.221 |   255.065 |
| std   |   155.363 |   154.896 |
| min   |     1     |     1     |
| 25%   |   115     |   120     |
| 50%   |   249     |   253.5   |
| 75%   |   388     |   391     |
| max   |   510     |   511     |

 Frecuencias de labels:
|    |   labels |   count |      pct |
|---:|---------:|--------:|---------:|
|  2 |        1 |    3882 | 24.1058  |
|  3 |        2 |    2512 | 15.5986  |
|

In [50]:
summarize_points_gt("data/patches-512-ol-160-m01/val_points_gt.csv")

archivo: data/patches-512-ol-160-m01/val_points_gt.csv

2183 filas - 4 columnas: [images, x, y, labels]

## Número de valores únicos por columna:
|        |    0 |
|:-------|-----:|
| images | 1166 |
| x      |  487 |
| y      |  464 |
| labels |    6 |

## Conteos de nulos por columna:
|    | index   |   0 |
|---:|:--------|----:|
|  0 | images  |   0 |
|  1 | x       |   0 |
|  2 | y       |   0 |
|  3 | labels  |   0 |

## Estadísticas de coordenadas de puntos (x, y)
|       |        x |        y |
|:------|---------:|---------:|
| count | 2183     | 2183     |
| mean  |  250.228 |  253.618 |
| std   |  152.142 |  151.559 |
| min   |    2     |    2     |
| 25%   |  120     |  123     |
| 50%   |  250     |  253     |
| 75%   |  379     |  380     |
| max   |  510     |  509     |

 Frecuencias de labels:
|    |   labels |   count |      pct |
|---:|---------:|--------:|---------:|
|  0 |        1 |     824 | 37.7462  |
|  3 |        2 |     225 | 10.3069  |
|  2 |        3 |     36